
# Système 2D avec isoclines, portrait de phase et intégrateurs (Euler, RK4, RK45)

On considère :
\[
\begin{cases}
\dot x = -r\,x + \phi(\lambda),\\[2mm]
\dot \lambda = (r+\rho)\,\lambda - U'(x),
\end{cases}
\qquad
U(x)=5\ln(1+x),\quad D(c)=\tfrac12(c-0.2)^2,\quad \phi=(D')^{-1}.
\]
Ici, \(D'(c)=c-0.2\Rightarrow \phi(\ell)=\ell+0.2\), et \(U'(x)=\frac{5}{1+x}\).

Le notebook trace :
- les **isoclines** \(\dot x=0\) et \(\dot \lambda=0\),
- le **portrait de phase** (champ de vecteurs),
- des **trajectoires** via `solve_ivp` (RK45), et via nos schémas **Euler explicite** et **RK4**,
- les **séries temporelles** \(x(t), \lambda(t)\) et la trajectoire \((x,\lambda)\).


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# ---------- paramètres ----------
r = 2.0
rho = 1.0

def U(x):
    return 5.0*np.log(1.0 + x)

def dU(x):
    return 5.0/(1.0 + x)

def D(c):
    return 0.5*(c - 0.2)**2

def dD(c):
    return (c - 0.2)

def phi(lam):
    # inverse de dD: c = lam + 0.2
    return lam + 0.2

def dphi(lam):
    return 1.0

# Champ de vecteurs du système
def f_vec(t, y):
    x, lam = y
    return np.array([ -r*x + phi(lam),
                      (r+rho)*lam - dU(x) ])

# isoclines:
# xdot=0 => -r x + phi(l)=0  => l = phi^{-1}(r x) = r x - 0.2
def nullcline_x(x):
    return r*x - 0.2

# lamdot=0 => (r+rho) l - dU(x) = 0 => l = dU(x)/(r+rho)
def nullcline_lam(x):
    return dU(x)/(r+rho)

print("Check φ'(ℓ)=", dphi(0.0), "   U'(x) at x=0:", dU(0.0))


## Isoclines & portrait de phase

In [ ]:

# Domaine d'affichage
xmin, xmax = 0.0, 4.0
ymin, ymax = 0.0, 4.0

# Courbes d'isoclines
xx = np.linspace(xmin, xmax, 400)
lam_nc_x   = nullcline_x(xx)
lam_nc_lam = nullcline_lam(xx)

plt.figure()
plt.plot(xx, lam_nc_x, label=r'$\dot x=0:\ \lambda=r\,x-0.2$')
plt.plot(xx, lam_nc_lam, label=r'$\dot\lambda=0:\ \lambda=U^\prime(x)/(r+\rho)$')
plt.xlim(xmin, xmax); plt.ylim(ymin, ymax)
plt.xlabel('x'); plt.ylabel(r'$\lambda$')
plt.title('Isoclines')
plt.legend()
plt.show()

# Champ de vecteurs (portrait de phase)
xv = np.linspace(xmin, xmax, 20)
yv = np.linspace(ymin, ymax, 20)
X, Y = np.meshgrid(xv, yv)
Ufield = -r*X + phi(Y)
Vfield = (r+rho)*Y - dU(X)

plt.figure()
plt.quiver(X, Y, Ufield, Vfield)
plt.xlim(xmin, xmax); plt.ylim(ymin, ymax)
plt.xlabel('x'); plt.ylabel(r'$\lambda$')
plt.title('Portrait de phase (champ de vecteurs)')
plt.show()


## Intégration avec `solve_ivp` (RK45) — séries temporelles

In [ ]:

T = 10.0
t_eval = np.linspace(0.0, T, 400)
y0 = np.array([1.0, 1.0])  # conditions initiales

sol = solve_ivp(f_vec, (0.0, T), y0, method='RK45', t_eval=t_eval)

plt.figure()
plt.plot(sol.t, sol.y[0], label='x(t)')
plt.plot(sol.t, sol.y[1], label=r'$\lambda(t)$')
plt.xlabel('t'); plt.title('Séries temporelles — RK45')
plt.legend(); plt.show()


## Schémas maison : Euler explicite et RK4

In [ ]:

def euler_explicit(f, y0, N, T):
    t = np.linspace(0.0, T, N+1)
    h = t[1] - t[0]
    y = np.zeros((N+1, len(y0)), dtype=float)
    y[0] = y0
    for n in range(N):
        y[n+1] = y[n] + h * f(t[n], y[n])
    return t, y

def rk4(f, y0, N, T):
    t = np.linspace(0.0, T, N+1)
    h = t[1] - t[0]
    y = np.zeros((N+1, len(y0)), dtype=float)
    y[0] = y0
    for n in range(N):
        tn, yn = t[n], y[n]
        k1 = f(tn, yn)
        k2 = f(tn + 0.5*h, yn + 0.5*h*k1)
        k3 = f(tn + 0.5*h, yn + 0.5*h*k2)
        k4 = f(tn + h,       yn + h*k3)
        y[n+1] = yn + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    return t, y

# Comparaison avec un même nombre de pas
T_demo = 5.0
N = 40
y0_demo = np.array([2.0, 0.5])

tE, yE = euler_explicit(f_vec, y0_demo, N, T_demo)
tR, yR = rk4(f_vec, y0_demo, N, T_demo)
sol_demo = solve_ivp(f_vec, (0.0, T_demo), y0_demo, method='RK45', t_eval=np.linspace(0.0, T_demo, 5*N+1))

plt.figure()
plt.plot(tE, yE[:,0], marker='+', linestyle='', label='Euler: x')
plt.plot(tE, yE[:,1], marker='+', linestyle='', label='Euler: λ')
plt.plot(tR, yR[:,0], marker='o', linestyle='', label='RK4: x')
plt.plot(tR, yR[:,1], marker='o', linestyle='', label='RK4: λ')
plt.plot(sol_demo.t, sol_demo.y[0], label='RK45: x (ref)')
plt.plot(sol_demo.t, sol_demo.y[1], label='RK45: λ (ref)')
plt.xlabel('t'); plt.title("Comparaison Euler / RK4 / RK45 (même horizon)")
plt.legend(); plt.show()


## Trajectoires dans le plan de phase \((x,\lambda)\)

In [ ]:

plt.figure()
plt.plot(yE[:,0], yE[:,1], label='Euler')
plt.plot(yR[:,0], yR[:,1], label='RK4')
plt.plot(sol_demo.y[0], sol_demo.y[1], label='RK45 (ref)')
plt.xlabel('x'); plt.ylabel(r'$\lambda$')
plt.title('Trajectoires dans le plan (x, λ)')
plt.legend(); plt.show()
